# 모델 2 - 기능 3에 대해 추가 처리 가능한 모델2
#### 모델 2는 모델 1에서 기능 3 지하철 트리거 인식 시 작동되는 모델임
#### 모델 1에서 기능 3으로 분류(카테고리 :3) 인 경우 해당 모델에서 역별 분류 작업 수행
#### 최종 수정일 : 20-05-27
* * *

* * *
## 3. 테스트
### 3.1 녹음 파일 생성

> [변경사항]
- 4초로 제한
* * *
> [기존과 동일]
- 블루투스 이어폰의 외부 마이크 사용 (채널 조정)

In [18]:
import librosa
import scipy.signal as signal
import numpy as np
import pandas as pd
import joblib

In [43]:
#환경 확인
import pyaudio
import wave
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 15
WAVE_OUTPUT_FILENAME = "test_file.wav"
audio = pyaudio.PyAudio()

In [17]:
# start Recording
stream = audio.open(format=pyaudio.paInt16,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    input_device_index=1,
                    frames_per_buffer=CHUNK)
print ("recording...")
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print ("finished recording")

recording...
finished recording


In [4]:
stream.stop_stream()
stream.close()
audio.terminate()
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

* * *
## 3. 테스트
### 3.2 전처리

> [변경사항]
- 모듈화 할 것
* * *
> [기존과 동일]


In [40]:
import numpy as np
import pandas as pd 
import glob

In [41]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [42]:
def parse_audio_files(filenames):
    rows = len(filenames)
    # feature는 각 파일 별 row(window) * 피처 의 2차원 행렬
    # labels은 파일 별 카테고리 int 값
    features = np.zeros((rows,193))
    i = 0
    for fn in filenames:
        try:
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        except:
            print("error : "+fn)
        else:
            features[i] = ext_features
            print("성공")
            i += 1
    return features

In [44]:
audio_files = []
audio_files.extend(glob.glob(WAVE_OUTPUT_FILENAME))

In [45]:
print(len(audio_files))

1


In [46]:
files = audio_files
X_test = parse_audio_files(files)

성공


* * *
## 3. 테스트
### 3.3 모델 적용

> [변경사항]

* * *
> [기존과 동일]
- 0 : 사이렌, 민방위 등 알림음
- 1,2 : 차량 경적, 엔진소리
- 3 : 환승 트리거

In [47]:
X_test = X_test.reshape(X_test.shape[0], 193, 1)

In [48]:
model = joblib.load('model_2.pkl')

In [49]:
pred = model.predict_proba(X_test)

In [50]:
print(pred)

[[20.17857]]


In [51]:
ans = round(float(pred))
de_label = pd.read_csv('de_train_label.csv', engine='python', index_col = None)
print("이번 역은 "+de_label['name'][ans]+"역 입니다.")

이번 역은 신도림역 입니다.
